In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np
import pandas as pd
from statistics import mean
from random import choice
%matplotlib inline
import matplotlib.pyplot as plt
import glob
from gendis.genetic import GeneticExtractor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tools.data_extraction import DataExtractor
from tools.trajectory_generator import TrajectoryGenerator
from tools.utils import standardize_data, set_movements, angle_diff,scale_down
from tools.experiments import Experiments

np.random.seed(1337)  # Random seed for reproducibility

tr_gen_options = {"samples": 25,
                  "freq": 3,
                  "reset_data": True}
dt_gen_options = {"n_test": 200}

train_test_options = {"split": 25}

df_csv_options = {"ts_class": "Bearing"}

gen_options = {"population_size": 25,
               "iterations": 20,
               "verbose": False,
               "normed": True,
               "add_noise_prob": 0.3,
               "add_shapelet_prob": 0.3,
               "wait": 10,
               "plot": None,
               "remove_shapelet_prob": 0.3,
               "crossover_prob": 0.66,
               "n_jobs": 4}

settings = {"trajectory_generator_options": tr_gen_options,
            "data_generation_options": dt_gen_options,
            "train_test_options":train_test_options,
            "define_csvs_option": df_csv_options,
            "genetic_options": gen_options}

tr_gen_options = settings["trajectory_generator_options"]
dt_gen_options = settings["data_generation_options"]
df_csvs_options = settings["define_csvs_option"]
train_test_options = settings["train_test_options"]
genetic_options = settings["genetic_options"]

if len(glob.glob('models')) == 0 :
    path = "models"
    try:
        os.mkdir(path)
    except OSError:  
        print ("Creation of the directory %s failed" % path)
    else:  
        print ("Successfully created the directory %s " % path)
else:
    files = glob.glob("models/*")
    for x in files:
        os.remove(x)



In [2]:
movement_list = ["step_up_left","step_up_right",
                 "spiral_movement_right","spiral_movement_left",
                 "expanding_square_right","expanding_square_left",
                 "creeping_line_left","creeping_line_right",
                 "sector_pattern_left","sector_pattern_right"]
all_movements = []
for x in movement_list:
    first_movement = [x]
    second_movement = ['random']
    movements = {'first_movement': first_movement,'second_movement': second_movement}
    all_movements.append(movements)


In [3]:
for x in all_movements:
    set_movements(x)
    # Create files if not created
    tr_gen = TrajectoryGenerator(**tr_gen_options)
    tr_gen.data_generation(**dt_gen_options)
    # Read in the datafiles
    dex = DataExtractor()
    train_df, test_df = dex.train_test_dataframes(**train_test_options)
#     print("The train samples length is:{0}".format(len(train_df[0] * train_test_options["split"]*2)))
#     print("The test samples length is:{0}\n".format(len(test_df[0] * train_test_options["split"]*2)))
    dex.define_csv(**df_csvs_options)
    
    x_train, y_train, x_test, y_test = dex.load_datasets()
    x_train=angle_diff(x_train)
    x_test=angle_diff(x_test)
    x_train, x_test = standardize_data(x_train, x_test)
    genetic_extractor = GeneticExtractor(**genetic_options)
    genetic_extractor.fit(x_train, y_train)
    distances_train = genetic_extractor.transform(x_train)
    lr = LogisticRegression()
    lr.fit(distances_train, y_train)
    from sklearn.externals import joblib
    filename = "models/"+x["first_movement"][0]+"_anglediff_logistic_regression.pkl"
    with open(filename, 'wb') as pickle_file:
        joblib.dump(lr, pickle_file)
        pickle_file.close()
    filename = "models/"+x["first_movement"][0]+"_anglediff_genetic_extractor.pkl"
    with open(filename, 'wb') as pickle_file:
        joblib.dump(genetic_extractor, pickle_file)
        pickle_file.close()
    
    

2019-05-27 20:27:25.832 | INFO     | tools.trajectory_generator:data_generation:623 - 
 Starting the generator with attributes: 
Original latitude: 37.295493
Original longitude: 23.824322
Initial bearing: 90
Initial speed: 10
Number of samples: 25
Starting time of measurements: 2015-02-01 12:00:00
With initial frequency of collected data: 3 min
and hard reset of data: True
2019-05-27 20:27:25.834 | INFO     | tools.trajectory_generator:data_generation:626 - Create directory 'generator_data' 
2019-05-27 20:27:25.840 | INFO     | tools.trajectory_generator:data_generation:632 - now creating data for movement: step_up_left
2019-05-27 20:27:42.091 | INFO     | tools.trajectory_generator:data_generation:637 - now creating data for movement: random
2019-05-27 20:28:22.770 | SUCCESS  | tools.trajectory_generator:data_generation:670 - Done with generator
2019-05-27 20:28:22.772 | INFO     | tools.data_extraction:read_datasets:22 - Reading the data files
2019-05-27 20:28:23.953 | SUCCESS  | too

2019-05-27 21:16:39.270 | INFO     | tools.trajectory_generator:data_generation:626 - Create directory 'generator_data' 
2019-05-27 21:16:39.273 | INFO     | tools.trajectory_generator:data_generation:632 - now creating data for movement: expanding_square_left
2019-05-27 21:16:43.429 | INFO     | tools.trajectory_generator:data_generation:637 - now creating data for movement: random
2019-05-27 21:17:24.960 | SUCCESS  | tools.trajectory_generator:data_generation:670 - Done with generator
2019-05-27 21:17:24.962 | INFO     | tools.data_extraction:read_datasets:22 - Reading the data files
2019-05-27 21:17:25.913 | SUCCESS  | tools.data_extraction:read_datasets:51 - Done reading files
2019-05-27 21:17:27.326 | INFO     | tools.data_extraction:define_csv:76 - Creating x_train.csv--y_train.csv and x_test.csv--y_test.csv 
2019-05-27 21:17:27.359 | SUCCESS  | tools.data_extraction:define_csv:101 - Done with train.csv
2019-05-27 21:17:27.380 | INFO     | tools.data_extraction:define_csv:122 - D